#task 5
1. Download Alice in Wonderland by Lewis Carroll from Project Gutenberg's website http://www.gutenberg.org/files/11/11-0.txt
2.	Perform any necessary preprocessing on the text, including converting to lower case, removing stop words, numbers / non-alphabetic characters, lemmatization.
3.	Find Top 10 most important (for example, in terms of TF-IDF metric) words from each chapter in the text (not "Alice"); how would you name each chapter according to the identified tokens?
4.	Find the Top 10 most used verbs in sentences with Alice. What does Alice do most often?
5.	*(not necessary) Find Top 100 most used verbs in sentences with Alice. Get word vectors using a pre-trained word2vec model and visualize them. Compare the words using embeddings. 


In [ ]:
import numpy as np
import pandas as pd
import re
import sys
import io
import urllib.request
import nltk
nltk.download("stopwords")
from nltk.tokenize import WhitespaceTokenizer
from nltk.tokenize import TreebankWordTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer

from sklearn.feature_extraction.text import TfidfVectorizer
from nltk import FreqDist
import gensim

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Download the text Alice in Wonderland

In [ ]:
Data="http://www.gutenberg.org/files/11/11-0.txt"

Perform any necessary preprocessing on the text, including converting to lower case, removing stop words, numbers / non-alphabetic characters, lemmatization.

In [ ]:
opener = urllib.request.URLopener({})
resource = opener.open(Data)
charset = resource.headers.get_content_charset()
print("Charset", charset)
text1 = resource.read()

if charset:
  text1 = text1.decode(resource.headers.get_content_charset())
else:
  text1 = text1.decode("utf-8")
text1[:1000]

Charset None


"\ufeffThe Project Gutenberg EBook of Alice’s Adventures in Wonderland, by Lewis Carroll\r\n\r\nThis eBook is for the use of anyone anywhere in the United States and most\r\nother parts of the world at no cost and with almost no restrictions\r\nwhatsoever.  You may copy it, give it away or re-use it under the terms of\r\nthe Project Gutenberg License included with this eBook or online at\r\nwww.gutenberg.org.  If you are not located in the United States, you'll have\r\nto check the laws of the country where you are located before using this ebook.\r\n\r\nTitle: Alice’s Adventures in Wonderland\r\n\r\nAuthor: Lewis Carroll\r\n\r\nRelease Date: June 25, 2008 [EBook #11]\r\n[Most recently updated: October 12, 2020]\r\n\r\nLanguage: English\r\n\r\nCharacter set encoding: UTF-8\r\n\r\n*** START OF THIS PROJECT GUTENBERG EBOOK ALICE’S ADVENTURES IN WONDERLAND ***\r\n\r\n\r\n\r\nProduced by Arthur DiBianca and David Widger\r\n\r\n[Illustration]\r\n\r\n\r\n\r\n\r\nAlice’s Adventures in Wonderl

In [ ]:
type(text1)

str

In [ ]:
text2 = re.sub('<.*?>', ' ', text1)
##text2 = re.sub(' +', ' ', text2)
text2 = re.sub(r'[^\w\s]', '', text2)
clean_pattern = re.compile("End of the Project Gutenberg EBook.*")
text2 =  re.sub(clean_pattern, "", text2.replace("\n", " ").replace("\r", " "))
text2 = re.sub(' +', ' ', text2)
text2[:100]

'The Project Gutenberg EBook of Alices Adventures in Wonderland by Lewis Carroll This eBook is for th'

In [ ]:
CHAP1= text2.find("CHAPTER I Down the RabbitHole Alice")
CHAP12= text2.rfind("CHAPTER II The Pool of Tears Curiouser and curiouser")
CHA1=text2[CHAP1:CHAP12]

CHAP2= text2.find("CHAPTER II The Pool of Tears Curiouser and curiouser")
CHAP22= text2.rfind("CHAPTER III A CaucusRace and a Long Tale They were indeed")
CHA2=text2[CHAP2:CHAP22]

CHAP3= text2.find("CHAPTER III A CaucusRace and a Long Tale They were indeed")
CHAP32= text2.rfind("CHAPTER IV The Rabbit Sends in a Little Bill It was the White Rabbit")
CHA3=text2[CHAP3:CHAP32]

CHAP4= text2.find("CHAPTER IV The Rabbit Sends in a Little Bill It was the White Rabbit")
CHAP42= text2.rfind("CHAPTER V Advice from a Caterpillar The Caterpillar and Alice")
CHA4=text2[CHAP4:CHAP42]

CHAP5= text2.find("CHAPTER V Advice from a Caterpillar The Caterpillar and Alice")
CHAP52= text2.rfind("CHAPTER VI Pig and Pepper For a minute or two")
CHA5=text2[CHAP5:CHAP52]

CHAP6= text2.find("CHAPTER VI Pig and Pepper For a minute or two")
CHAP62= text2.rfind("CHAPTER VII A Mad Tea-Party There was a table set out under")
CHA6=text2[CHAP6:CHAP62]

CHAP7= text2.find("CHAPTER VII A Mad TeaParty There was a table set out under")
CHAP72= text2.rfind("CHAPTER VIII The Queens CroquetGround A large rosetree stood")
CHA7=text2[CHAP7:CHAP72]

CHAP8= text2.find("CHAPTER VIII The Queens CroquetGround A large rosetree stood")
CHAP82= text2.rfind("CHAPTER IX The Mock Turtles Story You cant")
CHA8=text2[CHAP8:CHAP82]

CHAP9= text2.find("CHAPTER IX The Mock Turtles Story You cant")
CHAP92= text2.rfind("CHAPTER X The Lobster Quadrille The Mock Turtle sighed")
CHA9=text2[CHAP9:CHAP92]

CHAP10= text2.find("CHAPTER X The Lobster Quadrille The Mock Turtle sighed")
CHAP102= text2.rfind("CHAPTER XI Who Stole the Tarts The King and")
CHA10=text2[CHAP10:CHAP102]

CHAP11= text2.find("CHAPTER XI Who Stole the Tarts The King and")
CHAP112= text2.rfind("CHAPTER XII Alices Evidence Here cried Alice")
CHA11=text2[CHAP11:CHAP112]

CHAP12= text2.find("CHAPTER XII Alices Evidence Here cried Alice")
CHAP122= text2.rfind("End of Project Gutenbergs")
CHA12=text2[CHAP12:CHAP122]
LISTA = CHA1+CHA2+CHA3+CHA4+CHA5+CHA6+CHA7+CHA8+CHA9+CHA10+CHA11+CHA12

In [ ]:
def clean_text(text):
    nltk.download('wordnet')
    tokens = text.split(' ')
    tokens = [token.lower() for token in tokens]
    stop_words = stopwords.words('english')
    text = [token for token in tokens if token not in stop_words]

    lemmatizer = WordNetLemmatizer()
    stemmer = PorterStemmer()
    
    [lemmatizer.lemmatize(token) for token in text]
    [stemmer.stem(token) for token in text]
    return text

In [ ]:
CHAPTER1=clean_text(CHA1)
CHAPTER2=clean_text(CHA2)
CHAPTER3=clean_text(CHA3)
CHAPTER4=clean_text(CHA4)
CHAPTER5=clean_text(CHA5)
CHAPTER6=clean_text(CHA6)
CHAPTER7=clean_text(CHA7)
CHAPTER8=clean_text(CHA8)
CHAPTER9=clean_text(CHA9)
CHAPTER10=clean_text(CHA10)
CHAPTER11=clean_text(CHA11)
CHAPTER12=clean_text(CHA12)


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Pack

In [55]:
def distrib(text):
    fdist1 = FreqDist(text)
    text=fdist1.most_common(11)
    a=np.array(text)
    a=pd.DataFrame(a)
    a=a[0]
    a=a[1:11]
    a=np.array(a)
    title=' '.join(a)
    return text, title


In [56]:
text_1, title1=distrib(CHAPTER1)
text_2, title2=distrib(CHAPTER2)
text_3, title3=distrib(CHAPTER3)
text_4, title4=distrib(CHAPTER4)
text_5, title5=distrib(CHAPTER5)
text_6, title6=distrib(CHAPTER6)
text_7, title7=distrib(CHAPTER7)
text_8, title8=distrib(CHAPTER8)
text_9, title9=distrib(CHAPTER9)
text_10, title10=distrib(CHAPTER10)
text_11, title11=distrib(CHAPTER11)
text_12, title12=distrib(CHAPTER12)
text_1,title1

([('alice', 27),
  ('little', 15),
  ('think', 11),
  ('way', 11),
  ('like', 11),
  ('see', 10),
  ('thought', 8),
  ('could', 8),
  ('time', 8),
  ('one', 8),
  ('said', 8)],
 'little think way like see thought could time one said')

In [ ]:
from nltk import word_tokenize
from  nltk.text import ConcordanceIndex

Find the Top 10 most used verbs in sentences with Alice. What does Alice do most often?

In [57]:
orig_stdout = sys.stdout
out = io.StringIO()
sys.stdout = out
vebstext.concordance("Alice", lines = 387)
savetext = out.getvalue()
sys.stdout = orig_stdout

In [58]:
text2 = re.sub('<.*?>', ' ', savetext)
text2 = re.sub(r'[^\w\s]', '', text2)
clean_pattern = re.compile("End of the Project Gutenberg EBook.*")
text2 =  re.sub(clean_pattern, "", text2.replace("\n", " ").replace("\r", " "))
text2 = re.sub(' +', ' ', text2)
text2[:100]

'Displaying 385 of 385 matches idence CHAPTER I Down the RabbitHole Alice was beginning to get very t'

In [59]:
n=list(savetext)
savetext

'Displaying 385 of 385 matches:\nidence CHAPTER I Down the RabbitHole Alice was beginning to get very tired of s\nnd what is the use of a book thought Alice without pictures or conversations So\nso _very_ remarkable in that nor did Alice think it so _very_ much out of the w\nand looked at it and then hurried on Alice started to her feet for it flashed a\nhe hedge In another moment down went Alice after it never once considering how \nipped suddenly down so suddenly that Alice had not a moment to think about stop\nrds as she fell past it Well thought Alice to herself after such a fall as this\nusand miles down I think for you see Alice had learnt several things of this so\nhat Latitude or Longitude Ive got to Alice had no idea what Latitude was or Lon\ndown There was nothing else to do so Alice soon began talking again Dinahll mis\nt do cats eat bats I wonder And here Alice began to get rather sleepy and went \nand dry leaves and the fall was over Alice was not a bit hurt and she jumped

In [60]:
real=word_tokenize(savetext)

In [61]:
nltk.download('averaged_perceptron_tagger')
real2=nltk.pos_tag(real)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [62]:
word_tag_fd = nltk.FreqDist(real2)
real3=[wt[0] for (wt, _) in word_tag_fd.most_common() if wt[1] == 'VBD']

In [63]:
real4=real3[0:10]
real4

['said',
 'was',
 'had',
 'thought',
 'went',
 'looked',
 'did',
 'replied',
 'began',
 'got']